In [2]:
# Import libraries and dependencies
import api_calls
import datetime as dt
import pandas as pd
from alpaca.data import TimeFrame
from alpaca.data import BarSet
import json

In [5]:
# Call the alpaca api to get the data

# Set the ticker
ticker = "SPY"

# Set timeframe to '1D'
timeframe = TimeFrame.Day

# Set start and end datetimes of 1 year, between now and 30 days ago.
start_date = dt.datetime.now() - dt.timedelta(30)
end_date = dt.datetime.now()

# Get 1 month's worth of historical data for SPY
alpaca = api_calls.call_alpaca(ticker, timeframe, start_date, end_date)

# Parse the data
data = BarSet.json(alpaca)

# Convert the data to a json string
data = json.loads(data)

# Create a Pandas DataFrame
stock_prices = pd.DataFrame(data['data'][ticker])

# Display sample data
stock_prices

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,SPY,2023-03-01T05:00:00+00:00,395.410,396.69,393.380,394.74,99723518.0,699672.0,395.137160
1,SPY,2023-03-02T05:00:00+00:00,392.680,398.69,392.330,397.81,85442636.0,612955.0,395.427267
2,SPY,2023-03-03T05:00:00+00:00,399.710,404.45,399.030,404.19,90119952.0,645491.0,402.059203
3,SPY,2023-03-06T05:00:00+00:00,405.050,407.45,404.010,404.47,72801949.0,539788.0,405.429412
4,SPY,2023-03-07T05:00:00+00:00,404.420,404.67,397.625,398.27,108327251.0,776810.0,400.379478
5,SPY,2023-03-08T05:00:00+00:00,398.390,399.71,396.590,398.92,74767148.0,578475.0,398.260115
6,SPY,2023-03-09T05:00:00+00:00,399.740,401.48,390.530,391.56,111965492.0,849860.0,395.433274
7,SPY,2023-03-10T05:00:00+00:00,390.990,393.16,384.320,385.91,189252996.0,1386973.0,388.308446
8,SPY,2023-03-13T04:00:00+00:00,381.810,390.39,380.650,385.36,157847796.0,1391476.0,386.088209
9,SPY,2023-03-14T04:00:00+00:00,390.500,393.45,387.050,391.73,149772836.0,1033031.0,390.217653


In [20]:
# Call the news api to get the data

# Set the ticker
ticker = "AAPL"

# Call the news api
news = api_calls.call_newsapi("+" + ticker + " ticker", start_date, end_date)

# Save the total number of articles
total_articles = news['totalResults']
print(total_articles)

# Create a Pandas DataFrame
stock_news = pd.DataFrame(news['articles'])

# Display sample data
stock_news

72


,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'MarketWatch'}",Philip van Doorn,Deep Dive: These stocks fared the worst as Pow...,"Bank stocks were hit hard, as investors braced...",https://www.marketwatch.com/story/these-stocks...,https://images.mktw.net/im-738118/social,2023-03-07T21:59:00Z,Federal Reserve Chairman Jerome Powell set the...
1,"{'id': None, 'name': 'MarketWatch'}",Barbara Kollmeyer,Need to Know: SVB’s rescue means the Fed won’t...,One casualty of regulators rescuing SVB deposi...,https://www.marketwatch.com/story/svbs-rescue-...,https://images.mktw.net/im-741731/social,2023-03-13T09:46:00Z,After a frantic regulators-to-the-rescue weeke...
2,"{'id': None, 'name': 'Yahoo Entertainment'}",William Dahl,You'll Never Guess the Top-Performing Stock of...,If a time machine could take you back to the s...,https://finance.yahoo.com/news/youll-never-gue...,https://media.zenfs.com/en/Benzinga/81290aab17...,2023-03-14T17:31:52Z,If a time machine could take you back to the s...
3,"{'id': None, 'name': 'Barron's'}",Ben Levisohn,The U.S. Is Scaring Off Foreign Investors. Is ...,For those with no interest in democracy and wi...,https://www.barrons.com/articles/us-global-inv...,https://images.barrons.com/im-750059/social,2023-03-25T18:06:00Z,Welcome to America the Uninvestible. That seem...
4,"{'id': None, 'name': 'Barron's'}",Andrew Bary,"Apple, Berkshire, Alphabet, and Microsoft Are ...",Companies such as Apple and Berkshire Hathaway...,https://www.barrons.com/articles/apple-alphabe...,https://images.barrons.com/im-736231/social,2023-03-06T13:57:00Z,Rich corporations are getting richer thanks to...
...,...,...,...,...,...,...,...,...
67,"{'id': None, 'name': 'Barron's'}",Eric J. Savitz,U.S. Companies Are Sitting on $3.6 Trillion in...,"When it comes to cash, Silicon Valley CFOs rec...",https://www.barrons.com/articles/svb-roku-corp...,https://images.barrons.com/im-745478/social,2023-03-17T06:08:55Z,As Silicon Valley Bank slid into receivership ...
68,"{'id': None, 'name': 'MarketWatch'}",Steve Goldstein,Need to Know: There’s another looming cliff — ...,There's another worry on the horizon: the loom...,https://www.marketwatch.com/story/theres-anoth...,https://images.mktw.net/im-751434/social,2023-03-27T10:37:46Z,"Is the banking crisis over? Well, famous last ..."
69,"{'id': None, 'name': 'Barron's'}",Teresa Rivas,Which Retailers Have the Magic? This Analyst K...,Oliver Chen of TD Cowen is cautiously optimist...,https://www.barrons.com/articles/retail-stocks...,https://images.barrons.com/im-748227/social,2023-03-24T05:36:25Z,Retailers are adjusting to a postpandemic worl...
70,"{'id': None, 'name': 'Barron's'}",Eric J. Savitz,Silicon Valley Confronts the End of Growth. It...,"With customers reining in spending, and regula...",https://www.barrons.com/articles/in-techs-new-...,https://images.barrons.com/im-735920/social,2023-03-04T00:45:00Z,Silicon Valley could use a reboot. The biggest...
